In [2]:
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [25]:
def comment_scraper(vids,numcomments):

    #sets up chrome
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--mute-audio")
    driver = webdriver.Chrome(options = chrome_options)

    #sets up return df
    result = pd.DataFrame()

    for vid in vids:

        #opens the video and stops the video
        driver.get(vid)
        time.sleep(2)
        ActionChains(driver).send_keys(" ").perform()
        
        #scrolls to the bottom of the page
        wait = WebDriverWait(driver,5)
        for _ in range(30): 
            ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
            time.sleep(0.2)


        #Gathers Data and places into a data frame
        comment = driver.find_elements_by_xpath('//*[@id="content-text"]')
        
        #gathers at minimum numcomments amount of comments
        while len(comment) <= numcomments:
            prevcomment = len(comment)
            
            for _ in range(10): 
                ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
                time.sleep(0.2)
                
            comment = driver.find_elements_by_xpath('//*[@id="content-text"]')
            
            if len(comment) == prevcomment:
                break
            
            
        #gathers exactly numcomments amount of comments
        comment = [i.text for i in comment]
        likes = [i.text for i in driver.find_elements_by_xpath('//*[@id="vote-count-middle"]')]
        
        if(len(comment) > numcomments):
            comment = comment[0:numcomments]
            likes = likes[0:numcomments]
            
        commentvid = pd.DataFrame(list(zip(comment,likes)), columns = ["Comment","Comment Likes"])
        
        
        commentvid["Video Name"] = driver.find_element_by_xpath('//*[@id="container"]/h1/yt-formatted-string').text
        commentvid["Video URL"] = vid
        commentvid["Video Likes"] = driver.find_element_by_xpath('//*[@id="top-level-buttons"]/ytd-toggle-button-renderer[1]').text
        commentvid["Video Dislikes"] = driver.find_element_by_xpath('//*[@id="top-level-buttons"]/ytd-toggle-button-renderer[2]').text
        commentvid["Channel"] = driver.find_element_by_xpath('//*[@id="text"]/a').text
        commentvid["Video Views"] = driver.find_element_by_xpath('//*[@id="count"]/yt-view-count-renderer/span[1]').text
        commentvid["Posted Date"] = driver.find_element_by_xpath('//*[@id="date"]/yt-formatted-string').text
        commentvid["Channel Subscribers"] = driver.find_element_by_xpath('//*[@id="owner-sub-count"]').text
    
        #appends the data 
        commentvid = commentvid[["Channel","Channel Subscribers","Video Name","Posted Date","Video Views","Video Likes","Video Dislikes","Comment","Comment Likes"]]
        result = result.append(commentvid, ignore_index = True)
        
    driver.close()

    return result

In [27]:
videos = ['https://www.youtube.com/watch?v=eISCH39_Sm4','https://www.youtube.com/watch?v=SntU6-vxzWc']

df = comment_scraper(videos,1000)
df

,Channel,Channel Subscribers,Video Name,Posted Date,Video Views,Video Likes,Video Dislikes,Comment,Comment Likes
0,You Suck At Cooking,2.34M subscribers,6 Milkshakes - You Suck at Cooking (episode 111),"Jun 25, 2020","1,029,638 views",61K,734,Black Lives Matter: how you can help. Check ou...,4.3K
1,You Suck At Cooking,2.34M subscribers,6 Milkshakes - You Suck at Cooking (episode 111),"Jun 25, 2020","1,029,638 views",61K,734,Also known as a lactose intoleran’s devilish d...,5.5K
2,You Suck At Cooking,2.34M subscribers,6 Milkshakes - You Suck at Cooking (episode 111),"Jun 25, 2020","1,029,638 views",61K,734,I've been making milkshakes with just a fork f...,774
3,You Suck At Cooking,2.34M subscribers,6 Milkshakes - You Suck at Cooking (episode 111),"Jun 25, 2020","1,029,638 views",61K,734,Just imagine him going up to his girlfriend an...,755
4,You Suck At Cooking,2.34M subscribers,6 Milkshakes - You Suck at Cooking (episode 111),"Jun 25, 2020","1,029,638 views",61K,734,The only internet cook who makes me question w...,242
...,...,...,...,...,...,...,...,...,...
1995,Binging with Babish,7.33M subscribers,French Toast | Basics with Babish,"Jun 18, 2020","2,879,542 views",115K,1.2K,I hate this background music ever since it fir...,1
1996,Binging with Babish,7.33M subscribers,French Toast | Basics with Babish,"Jun 18, 2020","2,879,542 views",115K,1.2K,I’m telling you use a quad fold dry towel inst...,
1997,Binging with Babish,7.33M subscribers,French Toast | Basics with Babish,"Jun 18, 2020","2,879,542 views",115K,1.2K,Damn you-- I had to make French toast this mor...,
1998,Binging with Babish,7.33M subscribers,French Toast | Basics with Babish,"Jun 18, 2020","2,879,542 views",115K,1.2K,French Toast Is SOOO UNDERRATED TBH 🤤 💕,


# What it is currently Capable Of
- Gathers Channel/Channel Subscribers/ Video Title/ Date Posted/ Video Views/ Video Likes/ Video Dislikes/ Comments/ Comment LIkes


# What is NOT Tested/ WIll be Implemented
- Videos with No comments
- Videos with disabled: comments, likes/dislikes
- Subscribers, Video Views, video LIkes, Video Dislikes, Comment Likes are in String not int

# What it Does Not Do
- collect comment replies



In [28]:
df.to_csv("comments.csv")